In [1]:
from datasets import Dataset, load_dataset
import json

In [2]:
# JSON → HF Dataset
with open("/home/ubuntu/protolyze/datageneration/esm8M_500kdataset_k30_optimized.json") as f:
    records = json.load(f)           # assume a list of dicts
ds_json = Dataset.from_list(records)
ds_json.push_to_hub("protolyzee/esm8M_500kdataset_k30_optimized")


KeyError: 0

In [8]:
import json
from datasets import Dataset

with open("/home/ubuntu/protolyze/datageneration/esm35M_500kdataset_k30_optimized.json") as f:
    data = json.load(f)

# If `data` is a dict mapping IDs→records, convert to list:
if isinstance(data, dict):
    records = list(data.values())
elif isinstance(data, list):
    records = data
else:
    raise ValueError(f"Unexpected JSON top‐level type: {type(data)}")

# Now this should work:
ds_json = Dataset.from_list(records)
ds_json.push_to_hub("protolyze/esm35M_500kdataset_k30_optimized", create_pr=1)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/81.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/protolyze/esm35M_500kdataset_k30_optimized/commit/debd947871c6f41e5731d850d4117eda165c8200', commit_message='Upload dataset', commit_description='', oid='debd947871c6f41e5731d850d4117eda165c8200', pr_url='https://huggingface.co/datasets/protolyze/esm35M_500kdataset_k30_optimized/discussions/1', repo_url=RepoUrl('https://huggingface.co/datasets/protolyze/esm35M_500kdataset_k30_optimized', endpoint='https://huggingface.co', repo_type='dataset', repo_id='protolyze/esm35M_500kdataset_k30_optimized'), pr_revision='refs/pr/1', pr_num=1)

In [ ]:
# CSV → HF Dataset
ds_csv = Dataset.from_csv("my_table.csv")  # auto-parses header row
ds_csv.push_to_hub("your-username/my-csv-dataset", create_pr=1)

In [30]:
from pathlib import Path
from huggingface_hub import HfApi
from datasets import load_dataset

# 1) Log in with `huggingface-cli login` beforehand or set HF_TOKEN env var.
api = HfApi()
user = "protolyze"

# 2) Loop over your files
for path in Path("/home/ubuntu/protolyze/analysis/").glob("*"):
    time.sleep(2)
    name = path.stem                # e.g. "my_records" or "table"
    repo_id = f"{user}/{name}"

    # 3) Create the dataset repo (no error if it already exists)
    api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)

    # 4) Load it as a HF Dataset
    if path.suffix == ".json":
        # if JSON-lines use lines=True
        ds = load_dataset("json", data_files=str(path), split="train", lines=False)
    elif path.suffix == ".csv":
        ds = load_dataset("csv", data_files=str(path), split="train")
    else:
        continue

    # 5) Push to Hub
   
    ds.push_to_hub(repo_id)
    
    print(f"Pushed {path.name} → {repo_id}")


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/repos/create

In [12]:
from pathlib import Path
from huggingface_hub import HfApi
from datasets import load_dataset

# 1) Log in with `huggingface-cli login` beforehand or set HF_TOKEN env var.
api = HfApi()
user = "protolyze"

# 2) Fetch all of your existing dataset repo names
existing = {
    repo.repo_id.split("/")[-1]
    for repo in api.list_repos(token=api.token, repo_type="dataset")
}

# 3) Loop over your local files
for path in Path("/home/ubuntu/protolyze/analysis/").glob("*"):
    name = path.stem               # e.g. "esm8M_500kdataset_k30_optimized"
    repo_id = f"{user}/{name}"

    # 4) Create the repo only if it doesn't already exist
    if name not in existing:
        api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)
        print(f"Created new dataset repo: {repo_id}")
    else:
        print(f"Dataset repo already exists: {repo_id} (will overwrite)")

    # 5) Load local file into a HF Dataset object
    suffix = path.suffix.lower()
    if suffix == ".jsonl":
        ds = load_dataset("json", data_files=str(path), split="train", lines=True)
    elif suffix == ".json":
        ds = load_dataset("json", data_files=str(path), split="train", lines=False)
    elif suffix == ".csv":
        ds = load_dataset("csv",  data_files=str(path), split="train")
    else:
        print(f"Skipping unsupported file type: {path.name}")
        continue

    # 6) Push to Hub (this will overwrite existing content in the repo)
    ds.push_to_hub(repo_id, use_auth_token=True)
    print(f"Pushed {path.name} → {repo_id}")


AttributeError: 'HfApi' object has no attribute 'list_repos'

In [33]:
from pathlib import Path
import time
from huggingface_hub import HfApi
from datasets import load_dataset

# 1) Log in with `huggingface-cli login` beforehand or set HF_TOKEN env var.
api  = HfApi()
user = "protolyze"

# 2) Fetch your existing dataset repo names
existing = {
    ds.id.split("/")[-1]
    for ds in api.list_datasets(author=user, token=api.token)
}

def safe_create_repo(repo_id, max_retries=5):
    for attempt in range(max_retries):
        try:
            api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)
            print(f"✅ Created dataset repo: {repo_id}")
            return
        except Exception as e:
            # If it's an HTTP 429, retry with backoff
            status = getattr(e, "status_code", None)
            if status == 429 and attempt < max_retries - 1:
                wait = 2 ** attempt
                print(f"⏳ Rate limited creating {repo_id}, retrying in {wait}s…")
                time.sleep(wait)
            else:
                raise

def safe_push(ds, repo_id, max_retries=5):
    for attempt in range(max_retries):
        try:
            ds.push_to_hub(repo_id)
            print(f"🚀 Pushed data → {repo_id}")
            return
        except Exception as e:
            status = getattr(e, "status_code", None)
            if status == 429 and attempt < max_retries - 1:
                wait = 2 ** attempt
                print(f"⏳ Rate limited pushing {repo_id}, retrying in {wait}s…")
                time.sleep(wait)
            else:
                raise

# 3) Loop over your local files
for path in Path("/home/ubuntu/protolyze/analysis/").glob("*"):
    name    = path.stem
    repo_id = f"{user}/{name}"

    # 4) Create the repo only if missing
    if name not in existing:
        safe_create_repo(repo_id)
        existing.add(name)
    else:
        print(f"🔄 Repo already exists: {repo_id} (overwriting)")

    # 5) Load the file as a Dataset
    ext = path.suffix.lower()
    if ext == ".jsonl":
        ds = load_dataset("json", data_files=str(path), split="train", lines=True)
    elif ext == ".json":
        ds = load_dataset("json", data_files=str(path), split="train", lines=False)
    elif ext == ".csv":
        ds = load_dataset("csv",  data_files=str(path), split="train")
    else:
        print(f"⚠️ Skipping unsupported file: {path.name}")
        continue

    # 6) Push (will overwrite existing data)
    safe_push(ds, repo_id)


🔄 Repo already exists: protolyze/simulator_finetune (overwriting)
⚠️ Skipping unsupported file: simulator_finetune
🔄 Repo already exists: protolyze/steering_experiment_1 (overwriting)
⚠️ Skipping unsupported file: steering_experiment_1.png
🔄 Repo already exists: protolyze/__pycache__ (overwriting)
⚠️ Skipping unsupported file: __pycache__
🔄 Repo already exists: protolyze/gravy_masked_steering (overwriting)
⚠️ Skipping unsupported file: gravy_masked_steering.png
🔄 Repo already exists: protolyze/combined_steering_experiments_custom_titles (overwriting)
⚠️ Skipping unsupported file: combined_steering_experiments_custom_titles.png
🔄 Repo already exists: protolyze/simulator (overwriting)
⚠️ Skipping unsupported file: simulator.py
🔄 Repo already exists: protolyze/semantic_search (overwriting)
⚠️ Skipping unsupported file: semantic_search.py
🔄 Repo already exists: protolyze/line_plots_by_label_combo (overwriting)
⚠️ Skipping unsupported file: line_plots_by_label_combo.png
🔄 Repo already exist

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/repos/create